In [1]:
import pandas as pd
import numpy as np
import os

Key meta data

In [2]:
root_path = "D:/GeoData/"

Land Registry Data

In [3]:
LReg_Names = ['Transaction_unique_identifier', 'Price', 'Date_of_Transfer', 'Postcode', 'Property_Type', 
              'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 
              'PPDCategory_Type', 'Record_Status_monthly_file_only']

LReg_Data2020 = pd.read_csv(root_path + "LandReg/pp-2020.csv", names = LReg_Names) 
LReg_Data2021 = pd.read_csv(root_path + "LandReg/pp-2021.csv", names = LReg_Names) 

LReg_Data = pd.concat([LReg_Data2020,LReg_Data2021])

NaPTAN Data

In [4]:
np_rail_names_kp = ['TiplocCode', 'CrsCode', 'StationName', 'Easting', 'Northing']
np_ferry_names_kp = ['FerryCode', 'Name', 'Easting', 'Northing']
np_bus_names_kp = ['CommonName', 'Easting', 'Northing']

RailwayStations = pd.read_csv(root_path + "NaPTANcsv/RailReferences.csv", usecols=np_rail_names_kp) 
FerryTerminals = pd.read_csv(root_path + "NaPTANcsv/FerryReferences.csv", usecols=np_ferry_names_kp) 
BusStops = pd.read_csv(root_path + "NaPTANcsv/Stops.csv", usecols=np_bus_names_kp, encoding = "ISO-8859-1") 

Road Accidents Data

In [16]:
Road_Accidents_names_kp = ['accident_year', 'accident_reference', 'location_easting_osgr', 'location_northing_osgr', 'police_force', 'accident_severity', 'number_of_vehicles', 'number_of_casualties', 'date', 'day_of_week', 'time']

Road_Accidents = pd.read_csv(root_path + "RoadSafety/dft-road-casualty-statistics-accident-2020.csv", usecols=Road_Accidents_names_kp, low_memory=False) 

Some Cleaning

In [5]:
#Make the date into a date
LReg_Data['Date_of_Transfer'] = pd.to_datetime(LReg_Data['Date_of_Transfer'], errors='ignore')

#Calculate the year of transfer
LReg_Data["Year_Of_Transfer"] = pd.DatetimeIndex(LReg_Data['Date_of_Transfer']).year

#remove the spaces from the postcode
LReg_Data["Postcode_NS"] = LReg_Data["Postcode"].astype(str).str.replace(" ","")

#Cols to keep
keep_cols = ['Price', 'Date_of_Transfer', 'Year_Of_Transfer', 'Postcode_NS', 'Property_Type', 'Old_New', 'Duration']
LReg_Data = LReg_Data.loc[:,keep_cols]

National statistics udprn file (only for house locations)

In [6]:
keep_cols = ['uprn', 'gridgb1e', 'gridgb1n', 'pcds']
file = os.listdir("D:/GeoData/NSUL_OCT_2020/Data")
path = [os.path.join("D:/GeoData/NSUL_OCT_2020/Data", i) for i in file if ".csv" in i]

NSUL_Data = pd.concat([pd.read_csv(i,  usecols=keep_cols) for i in path])

NSUL_Data["Postcode_NS"] = NSUL_Data["pcds"].astype(str).str.replace(" ","")

house_count = NSUL_Data.loc[:, ['Postcode_NS', 'uprn']].groupby(['Postcode_NS']).size().reset_index(name='Houses')

National statistics postcode lookup file and references

In [7]:
NSPL_Dir = "NSPL_NOV_2020_UK/"
NSPL_Filename = "Data/NSPL_NOV_2020_UK.csv"

NSPL_Data = pd.read_csv(root_path + NSPL_Dir + NSPL_Filename, low_memory = False) 

#Not keeping all of the columns, for fairly arbitrary reasons
keep_cols = ['pcd', 'oseast1m', 'osnrth1m', 'osgrdind', 'oa11', 'cty', 'ced', 'laua', 'ward', 'hlthau', 'nhser', 'ctry', 
             'rgn', 'pcon', 'teclec', 'ttwa', 'pct', 'nuts', 'park', 'lsoa11', 'msoa11', 'ccg', 'bua11', 'buasd11', 
             'ru11ind', 'pfa', 'calncv', 'stp', 'imd']  

NSPL_Data = NSPL_Data.loc[:,keep_cols]

In [8]:
NSPL_Data["Postcode_NS"] = NSPL_Data["pcd"].astype(str).str.replace(" ","")

In [9]:
def getlookup(filename, keep):
    lookup = pd.read_csv(root_path + NSPL_Dir + "Documents/" + filename)
    lookup = lookup.loc[:,keep]
    return lookup

In [10]:
oa11 = getlookup('2011 Census Output Area Classification Names and Codes UK.csv', ['OAC11', 'Supergroup', 'Group', 'Subgroup'])
cty = getlookup('County names and codes UK as at 04_20_NSPL.csv', ['CTY20CD', 'CTY20NM'])
ced = getlookup('County Electoral Division names and codes EN as at 12_17.csv', ['CED17CD', 'CED17NM'])
laua = getlookup('LA_UA names and codes UK as at 04_20.csv', ['LAD20CD', 'LAD20NM'])
ward = getlookup('Ward names and codes UK as at 12_19.csv', ['WD19CD', 'WD19NM'])
hlthau = getlookup('HLTHAU names and codes UK as at 04_19.csv', ['HLTHAUCD', 'HLTHAUNM'])
nhser = getlookup('NHSER names and codes EN as at 04_19.csv', ['NHSER19CD', 'NHSER19NM'])
ctry = getlookup('Country names and codes UK as at 08_12.csv', ['CTRY12CD', 'CTRY12NM'])
rgn = getlookup('Region names and codes EN as at 12_10 (GOR).csv', ['GOR10CD', 'GOR10NM', 'GOR10NMW', 'GOR10CDO'])
pcon = getlookup('Westminster Parliamentary Constituency names and codes UK as at 12_14.csv', ['PCON14CD', 'PCON14NM'])
teclec = getlookup('TECLEC names and codes UK as at 12_16.csv', ['TECLECCD', 'TECLECNM', 'TECLECCDO'])
ttwa = getlookup('TTWA names and codes UK as at 12_11 v5.csv', ['TTWA11CD', 'TTWA11NM'])
pct = getlookup('PCT names and codes UK as at 04_19.csv', ['PCTCD', 'PCTNM', 'PCTCDO'])
nuts = getlookup('LAU2 names and codes UK as at 12_18.csv', ['LAU218CD', 'LAU218NM'])
park = getlookup('National Park names and codes GB as at 08_16.csv', ['NPARK16CD', 'NPARK16NM'])
lsoa11 = getlookup('LSOA (2011) names and codes UK as at 12_12.csv', ['LSOA11CD', 'LSOA11NM'])
msoa11 = getlookup('MSOA (2011) names and codes UK as at 12_12.csv', ['MSOA11CD', 'MSOA11NM'])
ccg = getlookup('CCG names and codes UK as at 04_20.csv', ['CCG20CD', 'CCG20NM', 'CCG20CDH', 'CCG20NMW'])
bua11 = getlookup('BUA_names and codes UK as at 12_13.csv', ['BUA13CD', 'BUA13NM'])
buasd11 = getlookup('BUASD_names and codes UK as at 12_13.csv', ['BUASD13CD', 'BUASD13NM'])
ru11ind = getlookup('Rural Urban (2011) Indicator names and codes GB as at 12_16.csv', ['RU11IND', 'RU11NM'])
pfa = getlookup('PFA names and codes GB as at 12_15.csv', ['PFA15CD', 'PFA15NM'])
calncv     = getlookup('CALNCV names and codes EN as at 07_18.csv', ['CALNCV18CD', 'CALNCV18NM'])
stp = getlookup('STP names and codes EN as at 04_20.csv', ['STP20CD', 'STP20NM', 'STP20CDH'])

In [11]:
NSPL_Data = NSPL_Data.merge(house_count, left_on='Postcode_NS', right_on='Postcode_NS', how='left')
NSPL_Data = NSPL_Data.merge(oa11, left_on='oa11', right_on='OAC11', how='left')
NSPL_Data = NSPL_Data.merge(cty, left_on='cty', right_on='CTY20CD', how='left')
NSPL_Data = NSPL_Data.merge(ced, left_on='ced', right_on='CED17CD', how='left')
NSPL_Data = NSPL_Data.merge(laua, left_on='laua', right_on='LAD20CD', how='left')
NSPL_Data = NSPL_Data.merge(ward, left_on='ward', right_on='WD19CD', how='left')
NSPL_Data = NSPL_Data.merge(hlthau, left_on='hlthau', right_on='HLTHAUCD', how='left')
NSPL_Data = NSPL_Data.merge(nhser, left_on='nhser', right_on='NHSER19CD', how='left')
NSPL_Data = NSPL_Data.merge(ctry, left_on='ctry', right_on='CTRY12CD', how='left')
NSPL_Data = NSPL_Data.merge(rgn, left_on='rgn', right_on='GOR10CD', how='left')
NSPL_Data = NSPL_Data.merge(pcon, left_on='pcon', right_on='PCON14CD', how='left')
NSPL_Data = NSPL_Data.merge(teclec, left_on='teclec', right_on='TECLECCD', how='left')
NSPL_Data = NSPL_Data.merge(ttwa, left_on='ttwa', right_on='TTWA11CD', how='left')
NSPL_Data = NSPL_Data.merge(pct, left_on='pct', right_on='PCTCD', how='left')
NSPL_Data = NSPL_Data.merge(nuts, left_on='nuts', right_on='LAU218CD', how='left')
NSPL_Data = NSPL_Data.merge(park, left_on='park', right_on='NPARK16CD', how='left')
NSPL_Data = NSPL_Data.merge(lsoa11, left_on='lsoa11', right_on='LSOA11CD', how='left')
NSPL_Data = NSPL_Data.merge(msoa11, left_on='msoa11', right_on='MSOA11CD', how='left')
NSPL_Data = NSPL_Data.merge(ccg, left_on='ccg', right_on='CCG20CD', how='left')
NSPL_Data = NSPL_Data.merge(bua11, left_on='bua11', right_on='BUA13CD', how='left')
NSPL_Data = NSPL_Data.merge(buasd11, left_on='buasd11', right_on='BUASD13CD', how='left')
NSPL_Data = NSPL_Data.merge(ru11ind, left_on='ru11ind', right_on='RU11IND', how='left')
NSPL_Data = NSPL_Data.merge(pfa, left_on='pfa', right_on='PFA15CD', how='left')
NSPL_Data = NSPL_Data.merge(calncv , left_on='calncv', right_on='CALNCV18CD', how='left')
NSPL_Data = NSPL_Data.merge(stp, left_on='stp', right_on='STP20CD', how='left')

In [12]:
NSPL_RawNames = ['OAC11', 'CTY20CD', 'CED17CD', 'LAD20CD', 'WD19CD', 'HLTHAUCD', 'NHSER19CD', 'CTRY12CD', 'GOR10CD', 
                 'PCON14CD', 'TECLECCD', 'TTWA11CD', 'PCTCD', 'LAU218CD', 'NPARK16CD', 'LSOA11CD', 'MSOA11CD', 'CCG20CD', 
                 'BUA13CD', 'BUASD13CD', 'RU11IND', 'PFA15CD', 'CALNCV18CD', 'STP20CD']

NSPL_LookupNames = ['Supergroup', 'Group', 'Subgroup', 'CTY20NM', 'CED17NM', 'LAD20NM', 'WD19NM', 'HLTHAUNM', 'NHSER19NM', 
                    'CTRY12NM', 'GOR10NM', 'GOR10NMW', 'GOR10CDO', 'PCON14NM', 'TECLECNM', 'TECLECCDO', 'TTWA11NM', 
                    'PCTNM', 'PCTCDO', 'LAU218NM', 'NPARK16NM', 'LSOA11NM', 'MSOA11NM', 'CCG20NM', 'CCG20CDH', 'CCG20NMW',
                    'BUA13NM', 'BUASD13NM', 'RU11NM', 'PFA15NM', 'CALNCV18NM', 'STP20NM', 'STP20CDH']

Base_Names = ["Postcode_NS", "oseast1m", "osnrth1m"]

NSPL_Data = NSPL_Data.loc[:,Base_Names + NSPL_RawNames + NSPL_LookupNames]

Combine the two

In [13]:
Combined = LReg_Data.merge(NSPL_Data, left_on = "Postcode_NS", right_on = "Postcode_NS", how='left')

Save data

In [17]:
NSPL_Data.to_pickle(root_path + 'WorkingData/' + 'NSPL_Data.pickle')
Combined.to_pickle(root_path + 'WorkingData/' + 'LandReg.pickle')
NSUL_Data.to_pickle(root_path + 'WorkingData/' + 'NSUL_Data.pickle')
BusStops.to_pickle(root_path + 'WorkingData/' + 'BusStops_Data.pickle')
FerryTerminals.to_pickle(root_path + 'WorkingData/' + 'FerryTerminals_Data.pickle')
RailwayStations.to_pickle(root_path + 'WorkingData/' + 'RailwayStations_Data.pickle')
Road_Accidents.to_pickle(root_path + 'WorkingData/' + 'Road_Accidents_Data.pickle')